# In what major TX City (Dallas, Houston, Austin, San Antonio) should you open a business and what type of business should it be?

#### In this project, we will analyze major Texas Cities to discover a lack of a certain type of business in one of the city to decide whether we should open a business of that type there or not.

In [4]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

print('Libraries imported.')

Libraries imported.


Create list of TX Cities

In [28]:
tx_cities_df = pd.DataFrame({'city': ['Austin','Dallas','Houston','San Antonio'],
                             'lats': [30.2672, 32.7767, 29.7604, 29.4241],
                             'lons': [-97.7431, -96.7970, -95.3698, -98.4936]})

In [29]:
tx_cities_df

,city,lats,lons
0,Austin,30.2672,-97.7431
1,Dallas,32.7767,-96.7970
2,Houston,29.7604,-95.3698
3,San Antonio,29.4241,-98.4936


### Foursquare API Information

In [52]:
CLIENT_ID = 'WPVYKKSG3BEGIX5T3QFJOPTURIVR0FIT40RSZF0R42OMRCGE' # your Foursquare ID
CLIENT_SECRET = 'BQ5U2MDOBZ5D5VYXODXQCGDYLY0IIPOCN45LNOSLVLXPFCJQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 500 # A default Foursquare API limit value

## Get venues for each city

In [53]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:
city_venues = getNearbyVenues(names=tx_cities_df['city'],
                               latitudes=tx_cities_df['lats'],
                               longitudes=tx_cities_df['lons'])

print('getNearbyVenues Done')

Austin
Dallas
Houston
San Antonio
getNearbyVenues Done


In [55]:
# Createing back up to not run API if need a fresh data frame

city_backup = city_venues[:]

In [56]:
city_venues.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Austin,30.2672,-97.7431,Truluck's Seafood Steak & Crab,30.266757,-97.744968,Seafood Restaurant
1,Austin,30.2672,-97.7431,Upstairs at Caroline,30.268810,-97.742310,Hotel
2,Austin,30.2672,-97.7431,The Driskill,30.267841,-97.741715,Hotel
3,Austin,30.2672,-97.7431,Firehouse Hostel and Lounge,30.268096,-97.741283,Lounge
4,Austin,30.2672,-97.7431,JuiceLand,30.266037,-97.742623,Juice Bar


## Check how many venues the API pulled for each city

In [57]:
city_venues[['City', 'Venue']].groupby('City').count()

,Venue
City,
Austin,100
Dallas,26
Houston,61
San Antonio,100


In [58]:
# Check unique categories
print('There are {} unique categories'.format(city_venues['Venue Category'].nunique()))

There are 104 unique categories


## Let's encode the categories for each city

In [61]:
# one hot encoding
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")
# add city column back to dataframe
city_onehot['City'] = city_venues['City'] 

# move city column to the first column
fixed_columns = [city_onehot.columns[city_onehot.columns.get_loc('City')]] + list(city_onehot.loc[:, city_onehot.columns != 'City'].columns)
city_onehot = city_onehot[fixed_columns]

# group cities together and get their weighted average score for each Venue
city_grouped = city_onehot.groupby('City').mean().reset_index()

city_grouped.head()

,City,Airport Service,American Restaurant,Aquarium,Art Gallery,Art Museum,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Bistro,Boutique,Breakfast Spot,Bridge,Burger Joint,Café,Cajun / Creole Restaurant,Chinese Restaurant,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Donut Shop,Empanada Restaurant,Farmers Market,Fast Food Restaurant,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lounge,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Outdoors & Recreation,Park,Pedestrian Plaza,Performing Arts Venue,Pharmacy,Piano Bar,Pizza Place,Plaza,Pool,Pub,Public Art,Resort,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Shipping Store,Southern / Soul Food Restaurant,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Sushi Restaurant,Taco Place,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio
0,Austin,0.000000,0.010000,0.000000,0.00,0.00,0.010000,0.000000,0.00,0.06,0.01,0.000000,0.000000,0.01,0.00,0.010000,0.010000,0.01,0.01,0.00,0.01,0.01,0.05,0.060000,0.010000,0.00,0.01,0.000000,0.000000,0.01,0.00,0.01,0.000000,0.010000,0.000000,0.010000,0.000000,0.03,0.010000,0.010000,0.000000,0.04,0.01,0.00,0.000000,0.010000,0.00,0.00,0.090000,0.020000,0.00,0.000000,0.000000,0.01,0.020000,0.000000,0.01,0.02,0.01,0.030000,0.010000,0.01,0.020000,0.000000,0.00,0.020000,0.01,0.020000,0.00,0.020000,0.01,0.000000,0.010000,0.00,0.010000,0.00,0.01,0.010000,0.000000,0.000000,0.00,0.01,0.00,0.01,0.01,0.000000,0.00000,0.020000,0.000000,0.000000,0.00,0.04,0.000000,0.040000,0.010000,0.010000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000,0.01,0.010000,0.01
1,Dallas,0.000000,0.038462,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.038462,0.038462,0.00,0.00,0.038462,0.038462,0.00,0.00,0.00,0.00,0.00,0.00,0.076923,0.000000,0.00,0.00,0.000000,0.038462,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.038462,0.000000,0.038462,0.00,0.00,0.00,0.038462,0.000000,0.00,0.00,0.269231,0.000000,0.00,0.038462,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.038462,0.038462,0.00,0.000000,0.00,0.000000,0.00,0.038462,0.00,0.000000,0.000000,0.00,0.038462,0.00,0.00,0.000000,0.038462,0.038462,0.00,0.00,0.00,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.00,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00
2,Houston,0.016393,0.016393,0.016393,0.00,0.00,0.016393,0.016393,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.049180,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.065574,0.032787,0.00,0.00,0.016393,0.000000,0.00,0.00,0.00,0.016393,0.016393,0.016393,0.016393,0.016393,0.00,0.000000,0.016393,0.000000,0.00,0.00,0.00,0.000000,0.016393,0.00,0.00,0.049180,0.016393,0.00,0.000000,0.016393,0.00,0.016393,0.016393,0.00,0.00,0.00,0.016393,0.032787,0.00,0.000000,0.000000,0.00,0.016393,0.00,0.016393,0.00,0.000000,0.00,0.016393,0.065574,0.00,0.032787,0.00,0.00,0.016393,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.016393,0.04918,0.032787,0.016393,0.032787,0.00,0.00,0.000000,0.016393,0.016393,0.016393,0.016393,0.032787,0.016393,0.016393,0.00,0.016393,0.00,0.016393,0.00
3,San Antonio,0.000000,0.040000,0.000000,0.01,0.01,0.000000,0.000000,0.01,0.03,0.00,0.000000,0.000000,0.00,0.01,0.020000,0.000000,0.00,0.00,0.01,0.00,0.00,0.06,0.030000,0.000000,0.01,0.00,0.000000,0.000000,0.00,0.01,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000

## Get 10 Most Common Values for Each City

In [71]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##########################################################################

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_most = pd.DataFrame(columns=columns)
city_venues_most['City'] = city_grouped['City']

for ind in np.arange(city_grouped.shape[0]):
    city_venues_most.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

city_venues_most.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Austin,Hotel,Bar,Coffee Shop,Cocktail Bar,Gay Bar,Steakhouse,Speakeasy,Food Truck,Lounge,New American Restaurant
1,Dallas,Hotel,Coffee Shop,Café,Boutique,Department Store,Performing Arts Venue,Plaza,Pool,Mobile Phone Shop,Mexican Restaurant
2,Houston,Coffee Shop,Park,Hotel,Sandwich Place,Burger Joint,Performing Arts Venue,Seafood Restaurant,Mediterranean Restaurant,Theater,Concert Hall
3,San Antonio,Hotel,Mexican Restaurant,Cocktail Bar,Seafood Restaurant,American Restaurant,Sandwich Place,Steakhouse,Pizza Place,Bar,Coffee Shop


## Get 10 Least Common Values for Each City

In [70]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=True)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##########################################################################

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Least Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Least Common Venue'.format(ind+1))

# create a new dataframe
city_venues_least = pd.DataFrame(columns=columns)
city_venues_least['City'] = city_grouped['City']

for ind in np.arange(city_grouped.shape[0]):
    city_venues_least.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

city_venues_least.head()

,City,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,9th Least Common Venue,10th Least Common Venue
0,Austin,Airport Service,Furniture / Home Store,Greek Restaurant,Grocery Store,Historic Site,History Museum,Ice Cream Shop,Indian Restaurant,Japanese Restaurant,Mobile Phone Shop
1,Dallas,Airport Service,Pharmacy,Pedestrian Plaza,Park,Outdoors & Recreation,Nightclub,Neighborhood,Music Venue,Museum,Movie Theater
2,Houston,Yoga Studio,Food Truck,French Restaurant,Furniture / Home Store,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Russian Restaurant,Historic Site
3,San Antonio,Airport Service,Mobile Phone Shop,Latin American Restaurant,Juice Bar,Jazz Club,Japanese Restaurant,Israeli Restaurant,Movie Theater,Indian Restaurant,Gym


### Let's take a look on the most and least venues in each city

In [72]:
city_top_bot_merge = city_venues_most.merge(city_venues_least, on='City',how='left')

In [134]:
city_topbot = city_top_bot_merge.transpose()[:]
city_topbot.columns = list(city_topbot.iloc[0,:])
city_topbot = city_topbot.drop('City', axis=0).reset_index()

In [135]:
city_topbot['type'] = np.where(city_topbot['index'].str.contains('Most'), 'most', 'least')

In [139]:
city_topbot

,index,Austin,Dallas,Houston,San Antonio,type
0,1st Most Common Venue,Hotel,Hotel,Coffee Shop,Hotel,most
1,2nd Most Common Venue,Bar,Coffee Shop,Park,Mexican Restaurant,most
2,3rd Most Common Venue,Coffee Shop,Café,Hotel,Cocktail Bar,most
3,4th Most Common Venue,Cocktail Bar,Boutique,Sandwich Place,Seafood Restaurant,most
4,5th Most Common Venue,Gay Bar,Department Store,Burger Joint,American Restaurant,most
5,6th Most Common Venue,Steakhouse,Performing Arts Venue,Performing Arts Venue,Sandwich Place,most
6,7th Most Common Venue,Speakeasy,Plaza,Seafood Restaurant,Steakhouse,most
7,8th Most Common Venue,Food Truck,Pool,Mediterranean Restaurant,Pizza Place,most
8,9th Most Common Venue,Lounge,Mobile Phone Shop,Theater,Bar,most
9,10th Most Common Venue,New American Restaurant,Mexican Restaurant,Concert Hall,Coffee Shop,most


In [143]:
city_most = city_venues_most.transpose()[:]
city_most.columns = list(city_most.iloc[0,:])
city_most = city_most.drop('City', axis=0).reset_index(drop=True)

In [146]:
city_most.reset_index(inplace=True)
city_most['place'] = city_most['index']+1 

In [148]:
city_most.drop('index',axis=1,inplace=True)

In [150]:
city_most

,Austin,Dallas,Houston,San Antonio,place
0,Hotel,Hotel,Coffee Shop,Hotel,1
1,Bar,Coffee Shop,Park,Mexican Restaurant,2
2,Coffee Shop,Café,Hotel,Cocktail Bar,3
3,Cocktail Bar,Boutique,Sandwich Place,Seafood Restaurant,4
4,Gay Bar,Department Store,Burger Joint,American Restaurant,5
5,Steakhouse,Performing Arts Venue,Performing Arts Venue,Sandwich Place,6
6,Speakeasy,Plaza,Seafood Restaurant,Steakhouse,7
7,Food Truck,Pool,Mediterranean Restaurant,Pizza Place,8
8,Lounge,Mobile Phone Shop,Theater,Bar,9
9,New American Restaurant,Mexican Restaurant,Concert Hall,Coffee Shop,10


Looking at the top 10 most venues on each of the cities, we see that Coffee Shop is top 3 for Austin, Dallas, Houston but # 10 in San Antonio, this may point that the Coffee Shop market is up and coming in San Antonio and has not yet been over saturated.

In [151]:
city_most.to_csv('C:/Users/eadame/Documents/Learning/IBM Data Science Cert/Capstone Project/city_most.csv')

In [154]:
city_topbot.to_csv('C:/Users/eadame/Documents/Learning/IBM Data Science Cert/Capstone Project/city_most_least.csv')